$20^{th}$ Apr 2021

In [676]:
import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

In [677]:
# df = pd.read_excel('/home/douso/Documents/TrendData/Results/COVID-19-Test_result_Reporting_01_01_2021.xlsx', header=1)

In [678]:
cols_of_interest = ['SAMPLE_NUMBER', 'AGE', 'AGE_UNIT_(DAYS/MONTHS/YEARS)', 'GENDER_(M/F)', 'OCCUPATION', 'NATIONALITY', 
           'COUNTY_OF_RESIDENCE', 'SUB_COUNTY_OF_RESIDENCE', 'HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N', 'TRAVEL_FROM', 
           'QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD', 'SYMTOMS_SHOWN_(COUGH;FEVER;ETC)', 'DATE_OF_SAMPLE_COLLECTION', 
           'DATE_SAMPLE_RECEIVED_IN_THE_LAB', 'RESULT', 'LAB_CONFIRMATION_DATE']

In [679]:
data_dir = '/home/douso/Documents/TrendData/Results'
files = os.listdir(data_dir)
files = sorted(files)

In [680]:
def get_trend_data(df, cols):
    df_new = df.copy()
    df_new.columns = [col.replace(' ', '_').upper() for col in df.columns]
    df_cln = df_new[df_new['SAMPLE_NUMBER'].notna()]
    return df_cln[cols]

In [681]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, 'DATE_SAMPLE_RECEIVED_IN_THE_LAB')
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [682]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_01_07_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_08_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-B.xlsx']

In [683]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', pd.NaT)
#         df.to_excel(f'/home/douso/Documents/TrendData/Results/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

In [684]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
#         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [685]:
headers = ['TESTING LAB', 'CASE ID', 'TYPE OF CASE (INITIAL/REPEAT)',
       'SAMPLE NUMBER', 'NAME', 'ID/PASSPORT NUMBER', 'AGE',
       'AGE UNIT (DAYS/MONTHS/YEARS)', 'GENDER (M/F)', 'PHONE NUMBER',
       'OCCUPATION', 'NATIONALITY', 'COUNTY OF RESIDENCE',
       'SUB COUNTY OF RESIDENCE', 'VILLAGE/ESTATE OF RESIDENCE', 'WARD',
       'COUNTY OF DIAGNOSIS', 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N',
       'TRAVEL FROM', 'CONTACT WITH CASE Y/N', 'CONFIRMED CASE NAME',
       'QUARANTINE FACILITY/HOSPITAL/HOMESTEAD', 'HAVE SYMPTOMS Y/N',
       'DATE OF ONSET OF SYMPTOMS', 'SYMTOMS SHOWN (COUGH;FEVER;ETC)',
       'SAMPLE TYPE (NP SWAB, OP SWAB, SERUM SPUTUM ETC)',
       'DATE OF SAMPLE COLLECTION', 'DATE SAMPLE RECEIVED IN THE LAB',
       'RESULT', 'LAB CONFIRMATION DATE']

In [686]:
# for file in keyerrors[:-1]:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     df_file.columns = headers
#     df_file.to_excel(f'{data_dir}/add_col2/{file}', startrow=1)

https://www.wqu.edu/The following 60 sheets, besides some June sheets (manually fixed earlier), lacked the `DATE_SAMPLE_RECEIVED_IN_THE_LAB` column, it was added for uniformity:

- 'COVID-19-Test_result_Reporting_01_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_01_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_02_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_10_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_11_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_12_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx'
- 'COVID-19-Test_result_Reporting_17_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_17_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_22_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_23_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_25_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_25_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_08_2020.xlsx'

In [687]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_01_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_02_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_10_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_16_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx',
#  'COVID-19-Test_result_Reporting_17_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_17_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_22_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_23_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_25_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_25_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_08_2020.xlsx']

In [688]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', np.nan)
#         df.to_excel(f'{data_dir}/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

The follwing sheets have missing values for the `RESULT` column:
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'

Updated column names of some of the Jun files ['GENDER_(M/F)', 'LAB_CONFIRMATION_DATE', 'RESULT']

In [689]:
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     if file.endswith('.xlsx'):
#         try:
#             df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#             df_data = get_trend_data(df_file, cols_of_interest)
#             df_raw = df_raw.append(df_data)
#             count += 1
#         except: 
#             if KeyError:
# #                 raise
#                 print(f'KeyError: {file}')
#             elif ValueError:
#                 print(f'ValueError: {file}')
#         finally: pass
#     pass
# print(f'Files processed: {count}')
# print(f'Files in the folder: {len(files)}')

### Travel History Missing - the HOSTORY misspelling

- COVID-19-Test_result_Reporting_09_09_2020.xlsx
- COVID-19-Test_result_Reporting_11_09_2020.xlsx
- COVID-19-Test_result_Reporting_12_09_2020.xlsx
- COVID-19-Test_result_Reporting_14_09_2020.xlsx
- COVID-19-Test_result_Reporting_15_09_2020.xlsx
- COVID-19-Test_result_Reporting_18_10_2020.xlsx
- Files processed: 249
- Files in the folder: 256

In [690]:
# keyerrors = []
# # df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [691]:
# files = ['COVID-19-Test_result_Reporting_09_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_09-2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_10_2020.xlsx']

In [692]:
# for file in files:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     (df_file.rename(columns={'HAS TRAVEL HISTORY(LAST 14 DAYS) Y/N': 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N'}, errors='raise').
#     to_excel(f'{data_dir}/rename_col/{file}', index=False, startrow=1))

In [693]:
df_raw = pd.DataFrame()
count = 0
for file in files:
    if file.endswith('.xlsx'):
        try:
            df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
            df_data = get_trend_data(df_file, cols_of_interest)
            df_raw = df_raw.append(df_data)
            count += 1
        except: 
            if KeyError:
#                 raise
                print(f'KeyError: {file}')
            elif ValueError:
                print(f'ValueError: {file}')
        finally: pass
    pass
print(f'Files processed: {count}')
print(f'Files in the folder: {len(files)}')

/home/douso/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Files processed: 276
Files in the folder: 278


In [694]:
df_raw.head()

,SAMPLE_NUMBER,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,COVC22771,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [695]:
os.getenv('PWD')

'/home/douso'

### **Clean Headers**

In [696]:
cols_rename = ['S_NUM', 'AGE', 'AGE_UNIT', 'GEND', 'OCCU', 'NAT', 'COUNT_RES', 'S_COUNT_RES', 'TRAV_HIST', 
                    'TRAV_FROM', 'QUAR_PLACE', 'SYMPS', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'RESULT', 'DT_CONF']

In [697]:
# df_raw.columns = cols_rename

### **Clean Sample Numbers**

In [698]:
def cov_names(x):
    x = x.upper().replace(' ', '')
    if len(x) == 8:
        x = x.replace('COVC', 'COVC0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVC00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVC000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVC0000')
    return x

In [699]:
df_raw.head()

,SAMPLE_NUMBER,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,COVC22771,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [700]:
df_raw.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx', 
              index=False, header=cols_rename)

In [701]:
# df_raw.head()

In [702]:
# df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

In [703]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] != 'Years'].AGE_UNIT.unique()

In [704]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Months']#.AGE_UNIT.unique()

In [705]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

In [706]:
df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

### **Clean Sample Name**

In [707]:
df_name = df_mrg_mtdata.assign(S_NUM=df_mrg_mtdata['S_NUM'].apply(lambda x: cov_names(x)))

In [708]:
df_name[df_name['S_NUM'].str.len() < 8]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9383,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9384,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
17053,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00


In [709]:
df_name[df_name['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9383,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9384,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
17053,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00
24273,NHRL-S001,42,Years,M,NaN,Kenya,Nairobi,NaN,No,NaN,NaN,NaN,18/4/2021,NaN,Positive,20/4/2021
24274,NHRL-S002,20,Years,F,NaN,Kenya,Kiambu,NaN,No,NaN,NaN,NaN,18/4/2021,NaN,Positive,20/4/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25478,KEM-21-03-94425,17,Years,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",04/03/2021,NaN,Positive,11/03/2021
25479,KEM-21-03-94736,16,Years,M,NaN,Kenya,Nairobi,Roysambu,No,NaN,NaN,NaN,12/03/2021,NaN,Positive,16/03/2021
25480,KEM-21-03-94766,22,Years,M,NaN,Kenya,Kiambu,Gatundu North,No,NaN,NaN,"Headache,cough",2021-10-03 00:00:00,NaN,Positive,15/03/2021
25481,KEM-21-03-95380,60,Years,F,NaN,Kenya,Kiambu,Kabete,No,NaN,NaN,NaN,18/03/2021,NaN,Positive,19/03/2021


### **Clean Age**

In [710]:
df_name[df_name['AGE_UNIT'] == 'Days']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [711]:
df_months = df_name[df_name['AGE_UNIT'] == 'Months']

In [712]:
df_months.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
569,COVC09555,11,Months,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02 00:00:00
1150,COVC02581,1,Months,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03 00:00:00
1578,COVC17631,10,Months,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03 00:00:00
2429,COVC18026,6,Months,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04 00:00:00
2586,COVC18183,19,Months,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04 00:00:00


In [713]:
df_mn2yrs = df_months.assign(AGE=df_months['AGE'].map(lambda x: round(x / 12, 1))).replace('Months', 'Years')

In [714]:
df_mn2yrs.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
569,COVC09555,0.9,Years,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02
1150,COVC02581,0.1,Years,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03
1578,COVC17631,0.8,Years,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03
2429,COVC18026,0.5,Years,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04
2586,COVC18183,1.6,Years,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04


In [715]:
df_name[df_name['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [716]:
df_not_months = df_name[df_name['AGE_UNIT'] != 'Months']

In [717]:
df_not_months.shape

(25411, 16)

In [718]:
df_mn2yrs.shape

(72, 16)

In [719]:
df_years = df_not_months.append(df_mn2yrs)

In [720]:
df_years.shape

(25483, 16)

In [721]:
df_years['AGE_UNIT'].unique()

array(['Years', nan, 'years', 'Nil'], dtype=object)

In [722]:
df_years[df_years['AGE_UNIT'] == 'NO PATIENT DATASHEET']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [723]:
df_years[df_years['AGE_UNIT'].str.contains('M', 'F') == True]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [724]:
df_years[df_years['AGE_UNIT'] == 'Nil']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
14681,COVC04083,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14682,COVC04084,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14683,COVC04085,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Positive,2020-07-17 00:00:00
14684,COVC04086,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14685,COVC04087,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14686,COVC04088,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14687,COVC04089,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14688,COVC04090,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14689,COVC04091,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14690,COVC04092,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00


In [725]:
df_years_drop_age_unit = df_years.drop('AGE_UNIT', axis=1)

In [726]:
df_years_rencol = df_years_drop_age_unit.rename(columns={'AGE': 'AGE_YRS'})

In [727]:
df_years_rencol.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [728]:
df_years_rencol['AGE_YRS'].unique()

array([58, 63, 53, 52, 41, 9, 6, 39, 23, 36, 33, 42, 34, 32, 57, 56, 27,
       20, nan, 44, 26, 31, 35, 28, 37, 54, 29, 43, 24, 38, 47, 49, 40,
       51, 50, 25, 30, 45, 46, 22, 21, 48, 55, 65, 64, 60, 18, 16, 10, 17,
       15, 8, 13, 5, 12, 84, 59, 62, 68, 312, 61, 19, 2, 86, 80, 69, 72,
       3, 4, 7, 74, 11, 67, 87, 3.6, 1, 14, 1.5, 70, 2.42,
       'None indicated', 73, 6.5, 2.5, 75, 97, 'Nil', 71, 89, 96, 82, 77,
       66, 'Not indicated', 743, 78, 76, 'None', 9.5, 'nil', 79, 81, 98,
       'none indicated', 90, 85, 91, 100, 101, 83, 92, 'not_indicated',
       5.5, 'Not Indicated', '37', '36', '40', '38', '29', 94, 105, 0.5,
       'not Indicated', '30', '44', '33', '27', '26', '51', '28',
       'None Indicated', 542, 0.8, 0.9, 0.1, 1.6, 3.7, 1.7, 0.2, 0.3, 2.8,
       0.7, 1.2, 2.1, 0.4, 4.5, 1.8, 2.9, 0.6], dtype=object)

In [729]:
[x for x in df_years_rencol['AGE_YRS'].unique() if isinstance(x, str)]

['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 '37',
 '36',
 '40',
 '38',
 '29',
 'not Indicated',
 '30',
 '44',
 '33',
 '27',
 '26',
 '51',
 '28',
 'None Indicated']

In [730]:
to_rep = ['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 'not Indicated',
 'None Indicated',
 'NO PATIENT DATASHEET'
]

In [731]:
df_years_repnan2float = df_years_rencol.assign(AGE_YRS=df_years_rencol['AGE_YRS'].replace(to_rep, np.nan).astype(float))

In [732]:
df_years_repnan2float

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23470,COVC17089,0.6,M,NaN,Kenya,Mombasa,LIKONI,No,NaN,NaN,CO,2020-10-22 00:00:00,2020-10-27 00:00:00,Negative,2020-10-30 00:00:00
24538,COVC22371,1.6,M,NaN,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,NaN,2020-11-25 00:00:00,2020-11-27 00:00:00,NaN,2020-11-27 00:00:00
24684,COVC23794,0.2,M,NaN,Kenya,Kiambu,GITHUNGURI,No,NaN,NaN,H,2021-05-02 00:00:00,2021-05-05 00:00:00,Negative,2021-05-06 00:00:00
24805,COVC23131,2.5,F,NaN,Kenya,Mombasa,MVITA,No,NaN,NaN,NaN,2021-02-08 00:00:00,2021-02-11 00:00:00,Negative,2021-02-11 00:00:00


### **Clean Gender**

In [733]:
df_years_repnan2float['GEND'].unique()

array(['M', 'F', 'f', 'Female', 'Male', 'F`', 'MALE ', 'MALE', nan, 'Nil',
       'T'], dtype=object)

In [734]:
df_years_repnan2float[df_years_repnan2float['GEND'].isna()].sort_values('S_NUM')

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9762,COVC00749,42.0,NaN,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
12727,COVC01037,30.0,NaN,Nil,Kenyan,Nairobi,Embaksi,N,NaN,Nil,Asymtomatic,Nil,2020-06-15 00:00:00,Neg,2020-06-15 00:00:00
18119,COVC04494,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
18120,COVC04495,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
18121,COVC04496,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
23764,COVC05236,NaN,NaN,NaN,NaN,Nyeri,NaN,NaN,NaN,NaN,NaN,2020-07-28 00:00:00,NaN,Negative,2020-07-31 00:00:00
23611,COVC05255,NaN,NaN,NaN,NaN,NaN,not_indicated,No,NaN,NaN,NaN,NaN,NaN,Negative,2020-07-31 00:00:00
12666,COVC13844,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00
12667,COVC13845,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00
12668,COVC13846,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00


In [735]:
df_years_repnan2float[df_years_repnan2float['GEND'] == 'Nil']

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
11090,COVC00749,42.0,Nil,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
16201,COVC01628,NaN,Nil,Nil,Nil,Nyeri,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


COVEs:
- COVC13844
- COVC13845
- COVC13846
- COVC13847
- COVC13848
- COVC13849
- COVC13850
- COVC13851
- COVC13852
- COVC13853
- COVC16256
- COVC16257

In [736]:
df_years_repgender = df_years_repnan2float.assign(GEND=df_years_repnan2float['GEND'].replace(['f', 'Female', 'F`', 'T'], 'F')
                                                 .replace(['Male', 'MALE ', 'MALE'], 'M'))

In [737]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan, 'Nil'], dtype=object)

In [738]:
df_years_repgender['GEND'].replace('Nil', np.nan, inplace=True)

In [739]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan], dtype=object)

### **Clean Occupation**

In [740]:
df_years_repgender['OCCU'].unique()

array(['HCW', 'CEC', 'Psychologist', ..., 'Contestant', 'Maintanace',
       'Child accompanying inmate mothe'], dtype=object)

In [741]:
df_years_repgender['OCCU'] = df_years_repgender['OCCU'].replace(['Nil','nil'], np.nan)

### **Clean Nationality**

In [742]:
df_years_repgender['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Kenyan', 'Indian',
       'Ghana', 'Zimbabwe', 'Rwanda', 'Mauritania', 'Denmark', 'Pakistan',
       'Tanzania', 'Canada', 'Congo, Democratic Republic of the',
       'Burundi', 'Italy', 'Australia', 'Colombia', 'Hungary', 'Djibouti',
       'Finland', 'Senegal', 'kenya', 'Yemen', 'Turkey', 'Philippines',
       'Nigeria', 'Japan', 'Ireland', 'Tunisia', 'Burkina Faso', 'Brazil',
       'Portugal', 'Cameroon', 'Bangladesh', 'Madagascar',
       "Cote d'Ivoire", 'Egypt', 'China', 'KENYAN', 'Greece', 'Bulgaria',
       'Nil', 'Switzerland', 'Mali', 'Eritrea', 'Austria', 'Malawi',
       'Congo, Republic of the', 'Sri Lanka', 'United States', 'Jordan',
       'Zambia'], dtype=object)

In [743]:
df_years_repnat = df_years_repgender.assign(NAT=df_years_repgender['NAT'].replace('Nil', np.nan)
                                            .replace(['United States', 'Indian'
                                                      , 'Congo, Democratic Republic of the'
                                                     , 'Congo, Republic of the'] 
                                                     , ['United States of America (USA)','India'
                                                        , 'Democratic Republic of the Congo',
                                                       'Republic of the Congo'])
                                            .map(lambda x: str(x).replace(x, 'Kenya') if ('enya' in str(x).lower()) else x))

In [744]:
df_years_repnat['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Ghana', 'Zimbabwe',
       'Rwanda', 'Mauritania', 'Denmark', 'Pakistan', 'Tanzania',
       'Canada', 'Democratic Republic of the Congo', 'Burundi', 'Italy',
       'Australia', 'Colombia', 'Hungary', 'Djibouti', 'Finland',
       'Senegal', 'Yemen', 'Turkey', 'Philippines', 'Nigeria', 'Japan',
       'Ireland', 'Tunisia', 'Burkina Faso', 'Brazil', 'Portugal',
       'Cameroon', 'Bangladesh', 'Madagascar', "Cote d'Ivoire", 'Egypt',
       'China', 'Greece', 'Bulgaria', 'Switzerland', 'Mali', 'Eritrea',
       'Austria', 'Malawi', 'Republic of the Congo', 'Sri Lanka',
       'Jordan', 'Zambia'], dtype=object)

### **Clean County of Res**

In [745]:
df_years_repnat.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [746]:
len(df_years_repnat['S_COUNT_RES'].str.lower().unique())

893

In [747]:
len((df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize())
     .unique()))

815

In [748]:
s_counties = (df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).replace(str(x), str(np.nan)) if ('indicate' in str(x)) else str(x).strip().capitalize())
              .replace('nan', np.nan))

In [749]:
# sorted([s for s in s_counties.unique()])

In [750]:
counties = (df_years_repnat['COUNT_RES']
                   .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize() if (isinstance(x, str)) else x))

In [751]:
counties.unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Nakuru', 'Samburu',
       'Mombasa', nan, 'Nyeri', 'Kisii', 'Tana river', 'Kitui', 'Wajir',
       'Nyandarua', 'Kirinyaga', 'Meru', 'Uasin gishu', 'Siaya',
       'Bungoma', 'Kakamega', 'Nandi', 'Kilifi', 'Kwale', 'Homabay',
       'Muranga', 'Uganda', 'Mandera', 'Kampala', 'Niarobi', 'Kisumu',
       'Badiere', 'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo',
       'Nil', 'Vihiga', 'Narok', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Migori', 'Bomet',
       'Nyamira'], dtype=object)

In [752]:
df_years_county = (df_years_repnat.assign(COUNT_RES=counties)).assign(S_COUNT_RES=s_counties)

In [753]:
df_years_county['COUNT_RES'].unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Nakuru', 'Samburu',
       'Mombasa', nan, 'Nyeri', 'Kisii', 'Tana river', 'Kitui', 'Wajir',
       'Nyandarua', 'Kirinyaga', 'Meru', 'Uasin gishu', 'Siaya',
       'Bungoma', 'Kakamega', 'Nandi', 'Kilifi', 'Kwale', 'Homabay',
       'Muranga', 'Uganda', 'Mandera', 'Kampala', 'Niarobi', 'Kisumu',
       'Badiere', 'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo',
       'Nil', 'Vihiga', 'Narok', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Migori', 'Bomet',
       'Nyamira'], dtype=object)

### **Clean Subcounty of Res

In [754]:
df_years_scounty = df_years_county.assign(S_COUNT_RES=s_counties)

### **Clean Travel History**

In [755]:
df_years_scounty.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [756]:
df_years_scounty['TRAV_HIST'].unique()

array(['No', nan, 'Yes', 'N', 'JH', 'no', 'NO ', 'Nil', 'YES', 'NONE ',
       'NONE', 'Y', 'y', 'yes'], dtype=object)

In [757]:
df_travl = df_years_scounty.assign(TRAV_HIST=df_years_scounty['TRAV_HIST'].replace([np.nan, 'N', 'no', 'NO', 'Nil', 'NONE ', 'NONE'], 'No')
           .replace(['YES', 'Y', 'y', 'yes'], 'Yes'))


In [758]:
df_travl['TRAV_FROM'].unique()

array([nan, 'ELDORET', 'NAIVASHA', 'NAIROBI;NYERI;NANYUKI', 'NIL',
       'Ruaka', 'Ngoingwa', 'Limuru', 'Thika', 'Embakasi', 'Uganda',
       'Dubai', 'Nil', 'Not indicated on form ', 'TORORO UGANDA',
       'UGANDA', 'AWASI', 'KIGALI', 'ENTEBE', 'NAIROBI', 'JINJA',
       'KISUMU', 'KAMPALA', 'KASESE', 'KILIFI COUNTY', 'KWALE COUNTY',
       'NYAHURURU; NANYUKI', 'TORORO', 'KAMPALA UGANDA', 'RWANDA',
       'JINJA UGANDA', 'KAJIADO', 'TANZANIA', 'NOT APPLICABLE ',
       'SOUTH SUDAN ', 'NYERI', 'KWALE;KILIFI', 'KAKAMEGA', 'KWALE',
       'Mombasa', 'Nairobi', 'Eldoret', 'Malindi', 'Uasin Gishu', 'N',
       'Machakos', 'No', 'KILIFI/NAIROBI', 'NANYUKI SAGANA',
       'THARAKA NITHI', 'SHIMBA HILLS', 'KILIFI', 'MACHAKOS',
       'Netherlands', 'TAVETA', 'Mombasa (Stayed in Mtwapa for 4 days)',
       'KINANGO MSAMBWENI', 'MERU', 'THIKA', 'MUHONIA', 'KASESE UGANDA',
       'KIRINYAGA', 'KIAMBU', 'DAR ES SAALAM', 'TAITA TAVETA', 'Tanzania',
       'not_indicated', 'ETHIOPIA', 'Nyeri to

In [759]:
df_travl2 = (df_travl.assign(TRAV_FROM=df_travl['TRAV_FROM'].replace(['nan','NIL', 'Nil', 'Not indicated on form '
                  , 'NOT APPLICABLE ', 'N', 'N', 'No', 'not_indicated', 'Not applicable', 'Not indicated'],np.nan)
                    .replace(['Mombasa (stayed in mtwapa for 4 days)', 'KINANGO MSAMBWENI', 'Nan'
                              , 'Mombasa (Stayed in Mtwapa for 4 days)'], 
                    ['Mtwapa', 'Kinangop;Msambweni', np.nan, 'Mtwapa'])
                    .apply(lambda x: str(x).replace('/', ';')
                    .replace(' and ', ';')
                    .replace('-', ';')
                    .replace(',', ';')
                    .replace(' ; ', ';')
                    .replace('; ', ';')
                    .strip().capitalize() if (isinstance(x, str)) else x)))

In [760]:
df_travl2['TRAV_FROM'].unique()



array([nan, 'Eldoret', 'Naivasha', 'Nairobi;nyeri;nanyuki', 'Ruaka',
       'Ngoingwa', 'Limuru', 'Thika', 'Embakasi', 'Uganda', 'Dubai',
       'Tororo uganda', 'Awasi', 'Kigali', 'Entebe', 'Nairobi', 'Jinja',
       'Kisumu', 'Kampala', 'Kasese', 'Kilifi county', 'Kwale county',
       'Nyahururu;nanyuki', 'Tororo', 'Kampala uganda', 'Rwanda',
       'Jinja uganda', 'Kajiado', 'Tanzania', 'South sudan', 'Nyeri',
       'Kwale;kilifi', 'Kakamega', 'Kwale', 'Mombasa', 'Malindi',
       'Uasin gishu', 'Machakos', 'Kilifi;nairobi', 'Nanyuki sagana',
       'Tharaka nithi', 'Shimba hills', 'Kilifi', 'Netherlands', 'Taveta',
       'Mtwapa', 'Kinangop;msambweni', 'Meru', 'Muhonia', 'Kasese uganda',
       'Kirinyaga', 'Kiambu', 'Dar es saalam', 'Taita taveta', 'Ethiopia',
       'Nyeri town', 'Malaba', 'Nakuru', 'Narok', 'Within kenya',
       'Tharakanithi', 'Nyahururu', 'Nairobi;kisumu', 'Nairobi;nyahururu',
       'Nakuru;meru', 'Busia', 'Bungoma', 'Nanyuki', 'Kisii',
       'Emali;maku

In [761]:
df_travl2.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


### **Clean Symptoms**

In [762]:
df_travl2['SYMPS'] = (df_travl2['SYMPS'].apply(lambda x: str(x).replace(str(x), str(np.nan)) if ('symtom' in str(x)) else str(x).replace('F/C', 'FC'))
                      .replace(['[=]','nan'], np.nan))

### **Clean Dates**

In [763]:
def my_date_parser(x):
    try:
        if type(x) == pd.Timestamp:
            x = x.strftime('%d-%m-%Y')
        elif type(x) != pd.Timestamp and type(x) != (int, float, pd.NaT):
            x = pd.to_datetime(x, errors='ignore', dayfirst=True).strftime('%d-%m-%Y')
        elif x in ['None indicated', 'NIL', 'Not indicated on form ',
                    'Nil', 'None indicated', 'Leaked Sample - Empty', 'nan']:
            x = pd.NaT
    except (ValueError, AttributeError):
        x = pd.NaT
    return x

In [764]:
df_dt1 = df_travl2.assign(DT_SAM_COLL=df_travl2['DT_SAM_COLL'].apply(lambda x: my_date_parser(x)))

In [765]:
df_dt2 = df_dt1.assign(DT_SAM_RECEP=df_dt1['DT_SAM_RECEP'].apply(lambda x: my_date_parser(x)))

In [766]:
df_dts_cln = df_dt2.assign(DT_CONF=df_dt2['DT_CONF'].apply(lambda x: my_date_parser(x)))

In [767]:
df_dts_cln.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,01-01-2021,01-01-2021,Negative,01-01-2021
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,31-12-2020,01-01-2021,Negative,01-01-2021
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,01-01-2021,01-01-2021,Negative,01-01-2021
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,02-01-2021,01-01-2021,Negative,01-01-2021
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,03-01-2021,01-01-2021,Positive,01-01-2021


### **Clean Results**

In [768]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', nan, 'neg', 'pos', 'negative', 'positive',
       'Neg', 'Pos', 'Inconclusive', 'MS', 'Positve', 'Positive '],
      dtype=object)

In [769]:
df_dts_cln['RESULT'].map(lambda x: x.strip().lower() if (isinstance(x, str)) else x).unique()

array(['negative', 'positive', nan, 'neg', 'pos', 'inconclusive', 'ms',
       'positve'], dtype=object)

In [770]:
def cln_results(x):
    if isinstance(x, str):
        x = x.strip().lower()
        if 'neg' in x:
            x = x.replace(x, 'Negative')
        elif 'pos' in x:
            x = x.replace(x, 'Positive')
        elif x == 'inconclusive':
            x = x.replace(x, 'Inconclusive')
    return x
        

In [771]:
df_dts_cln['RESULT'] = df_dts_cln['RESULT'].map(lambda x: cln_results(x))

In [772]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', nan, 'Inconclusive', 'ms'], dtype=object)

In [773]:
df_dts_cln.head(2)

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,01-01-2021,01-01-2021,Negative,01-01-2021
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,31-12-2020,01-01-2021,Negative,01-01-2021


### **Remove Duplicates**

In [774]:
df_cln_srt = df_dts_cln.sort_values(['S_NUM', 'DT_CONF'])

In [775]:
df_cln_srt[df_cln_srt.duplicated(['S_NUM', 'DT_CONF'], keep=False) == True]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9721,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
11049,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
9722,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
11050,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
9723,COVC00710,45.0,M,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25265,COVC23805,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
19631,COVC23806,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
25266,COVC23806,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
25352,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",04-03-2021,NaT,Positive,11-03-2021


In [776]:
df_cln_srt[df_cln_srt.duplicated(keep=False) == True]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9721,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
11049,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
9722,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
11050,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
9723,COVC00710,45.0,M,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,09-06-2020,11-06-2020,Negative,12-06-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25265,COVC23805,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
19631,COVC23806,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
25266,COVC23806,NaN,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,25-05-2021,25-05-2021,Negative,25-05-2021
25352,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",04-03-2021,NaT,Positive,11-03-2021


In [777]:
df_dedud = df_cln_srt.drop_duplicates(['S_NUM', 'DT_CONF'], keep='first')

In [778]:
df_dedud.shape

(24071, 15)

### **Export Cleaned Data**

In [779]:
df_dedud.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln.xlsx', index=False)

In [780]:
df_dedud[df_dedud['RESULT'] == 'Positive'].to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln-pos.xlsx', index=False)

In [781]:
df_dedud_pos = df_dedud[df_dedud['RESULT'] == 'Positive']

In [782]:
df_dedud[df_dedud['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9383,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
9384,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
25425,KEM-012-IMC-004-12773,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
25426,KEM-012-IMC-557-12780,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
25422,KEM-012-IMH-533-12756,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24306,NHRL-S034,45.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24307,NHRL-S035,30.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24308,NHRL-S036,30.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24309,NHRL-S037,19.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021


In [783]:
df_dedud[df_dedud['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9383,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
9384,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
25425,KEM-012-IMC-004-12773,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
25426,KEM-012-IMC-557-12780,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
25422,KEM-012-IMH-533-12756,NaN,NaN,NaN,NaN,NaN,Nan,No,NaN,NaN,NaN,NaT,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24306,NHRL-S034,45.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24307,NHRL-S035,30.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24308,NHRL-S036,30.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021
24309,NHRL-S037,19.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,23-04-2021,NaT,Positive,25-04-2021


11-09-2020

COVC9774
COVC9775
COVE20
COVE21
COVC9776
COVC9777


### **ID Misssing Data**

In [784]:
start = 0
df_missing = pd.DataFrame(columns=cols_rename)
for num in df_dedud['S_NUM'].sort_values():
#     curr_row = df_dts_cln[df_dts_cln['S_NUM']]
    try:
        curr = int(num.lstrip('COVC').lstrip('0'))
        
        if curr - start > 1:
            start_id = f'COVC{start}'
            cov_id = f'COVC{curr}'
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(start_id)])
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(cov_id)])
        start = curr
    except ValueError:
#         print(f'ValueError: {num}')
        pass

In [785]:
df_missing.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [786]:
(df_missing[['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF']]
 .sort_values('S_NUM')
 .drop_duplicates('S_NUM', keep='first')
 .to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-missingII.xlsx', index=False))

### **ID Missing Results**

In [787]:
(df_dts_cln[df_dts_cln['RESULT'].isin(['Positive', 'Negative']) == 
            False][['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF', 'RESULT']]
.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-results-missingII.xlsx', index=False))

### **Filter Samples Seq'd in seq10**

In [788]:
# df_waiting = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt.xlsx')
# df_ext_seq10 = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/run10_extractionplates.xlsx')

In [789]:
# df_waiting.head()

In [790]:
# df_ext_seq10.head()

In [791]:
# ext_list = list(df_ext_seq10['Sample_Name'])

In [792]:
# df_waiting11 = df_waiting[df_waiting['Sample_Name'].isin(ext_list) == False]

In [793]:
# df_waiting11.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt2.xlsx', index=False)

In [794]:
# plt.style.available

### **Get GSAID Data**

In [795]:
df_gsaid = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/metadata_for_gisaid_ouso.xlsx')

In [796]:
df_gsaid.head()

,Sequence_name,Date_of_Sample_Collection,dt_received,dt_tested,County_of_Residence,Gender,Patient Age,Sequencing Technology
0,COVC00854,11-06-2020,NaN,13-06-2020,Nairobi,F,23.0,Illumina MiSeq
1,COVC00893,10-06-2020,NaN,13-06-2020,Nairobi,M,31.0,Illumina MiSeq
2,COVC00962,08-06-2020,14-06-2020,14-06-2020,Nairobi,M,31.0,Illumina MiSeq
3,COVC00987,08-06-2020,14-06-2020,14-06-2020,Nairobi,M,32.0,Illumina MiSeq
4,COVC00998,08-06-2020,14-06-2020,14-06-2020,Nairobi,M,25.0,Illumina MiSeq


In [797]:
gsaid_seq_nam = list(df_gsaid['Sequence_name'])

In [798]:
df_gsaid_export = df_dedud[df_dedud['S_NUM'].isin(gsaid_seq_nam) == True]

In [799]:
df_gsaid_export.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/metadata_gisaid_ousoII.xlsx', index=False)

### **Metadata + Cts**

In [800]:
df_Cts = pd.read_excel('/home/douso/Documents/RunSheets/Cts/runsheet-cts.xlsx')
# df_nhrl_Cts = pd.read_excel('/home/douso/Documents/RunSheets/Cts/NHRL-cts.xlsx')
# df_Cts = df_covc_Cts.append(df_nhrl_Cts)

In [801]:
df_metCts = df_dedud.set_index('S_NUM').merge(df_Cts.set_index('Sample_Name'), how='outer', left_index=True, right_index=True).reset_index().rename(columns={'index': 'S_NUM'})

In [802]:
df_metCts.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF,Ct_Mean
0,COVC00001,35.0,F,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,02-06-2020,NaN
1,COVC00002,29.0,M,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,02-06-2020,NaN
2,COVC00003,50.0,M,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,02-06-2020,NaN
3,COVC00004,35.0,F,Casual,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,02-06-2020,NaN
4,COVC00005,27.0,F,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,02-06-2020,NaN


In [803]:
df_metCts.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-resultsCts-merged-cln.xlsx', index=False)

In [804]:
df_metCts.shape

(24108, 16)

In [805]:
df_metCts[df_metCts['RESULT'] == 'Positive'].to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-resultsCts-merged-cln-pos.xlsx', index=False)